# S03 T02: CNN Architecture

## The one with the libraries

In [ ]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras import utils
import pickle
import numpy as np
from keras import models

In [ ]:
path_save = "/content/drive/My Drive/Piclkes Mel Spec/Aquivos H5/"
path = "/content/drive/My Drive/Piclkes Mel Spec/Com aumentation/"

## The one who prepare the CNN input 

In [ ]:
def read_pickle(name):
    with (open(name, 'rb')) as openfile:
        while True:
            try:
                one_instance = pickle.load(openfile)
            except EOFError:
                break
    one_instance = np.asanyarray(one_instance)
    return one_instance

In [ ]:
x_train = read_pickle(path+'Treino_x_Treino_mel_aug.pickle')
y_train = read_pickle(path+'Treino_y_Treino_mel_aug.pickle')
x_test = read_pickle(path+'Teste_x_Teste_mel.pickle')
y_test = read_pickle(path+'Teste_y_Teste_mel_int.pickle')

In [ ]:
print(x_test.shape)
print(y_test.shape)
print(x_train.shape)
print(y_train.shape)

(4001, 201, 201)
(4001,)
(23211, 201, 201)
(23211,)


In [ ]:
def pre_processing_data_to_keras_format(x, y):
    x = x.reshape(x.shape[0],x.shape[1],x.shape[2],1)
    y = utils.to_categorical(y) #8 -> 0 0 0 0 0 0 0 0 1 0
    return x, y

In [ ]:
# Redimensionando os espectogramas e labels
x_train, y_train = pre_processing_data_to_keras_format(x_train, y_train)
x_test, y_test = pre_processing_data_to_keras_format(x_test, y_test)

In [ ]:
print(x_test.shape)
print(y_test.shape)
print(x_train.shape)
print(y_train.shape)
print(y_train[23000])

(4001, 201, 201, 1)
(4001, 13)
(23211, 201, 201, 1)
(23211, 13)
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]


## The one with the network parameters

In [ ]:
#Parâmetros de modelo
dropout_value = 0.5

## The one with the network architecture

In [ ]:
# Inicializando a CNN
classifier = Sequential()

#Camada de convolução
classifier.add(Convolution2D(32, kernel_size=(5,5), input_shape = (201, 201,1), activation = 'relu', padding='same', name = 'conv_1'))

#Camada de pooling
classifier.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2), padding='same', name = 'pool_1'))

#Camada de normalização
classifier.add(BatchNormalization())

#Segunda camada convolucional
classifier.add(Convolution2D(64, kernel_size=(5,5), activation = 'relu', padding='same', name = 'conv_2'))

#Segunda camada de pooling
classifier.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same', name = 'pool_2'))

#Segunda camada de normalização
classifier.add(BatchNormalization())

#Terceira camada convolucional
classifier.add(Convolution2D(64, kernel_size=(5,5), activation = 'relu', padding='same', name = 'conv_3'))

#Terceira camada de pooling
classifier.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same', name = 'pool_3'))

#Terceira camada de normalização
classifier.add(BatchNormalization())

#Quarta camada convolucional
classifier.add(Convolution2D(64, kernel_size=(5,5), activation = 'relu', padding='same', name = 'conv_4'))

#Quarta camada de pooling
classifier.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same', name = 'pool_4'))

#Quarta camada de normalização
classifier.add(BatchNormalization())

#Vetorizando os mapas de características do último pooling (camada de entrada)
classifier.add(Flatten())

#Dropout
classifier.add(Dropout(dropout_value))

#Camada totalmente conectada ou oculta
classifier.add(Dense(activation='relu', units=256, name = 'dense_1'))

#Dropout
classifier.add(Dropout(dropout_value))

#Camada de saída
classifier.add(Dense(activation='softmax', units=13,  name = 'classification'))

In [ ]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 201, 201, 32)      832       
_________________________________________________________________
pool_1 (MaxPooling2D)        (None, 101, 101, 32)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 101, 101, 32)      128       
_________________________________________________________________
conv_2 (Conv2D)              (None, 101, 101, 64)      51264     
_________________________________________________________________
pool_2 (MaxPooling2D)        (None, 51, 51, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 51, 51, 64)        256       
_________________________________________________________________
conv_3 (Conv2D)              (None, 51, 51, 64)        1

$Atividade (2.0 pt):$ apresentar um notebook com a arquitetura adaptada para os spectrograms da sua base.

# S03 T03: CNN training

In [ ]:
from keras.callbacks import ModelCheckpoint
from keras import optimizers

In [ ]:
#Parâmetros de treinamento
learning_rate = 0.001
decay_=0.0001
epochs = 5
batch_size = 128
validation_split=0.1

In [ ]:
#Compilando a CNN: inicializando as variáveis
sgd = optimizers.SGD(lr=learning_rate, decay=decay_)
classifier.compile(optimizer = 'sgd', loss= 'categorical_crossentropy', metrics=['accuracy'])

checkpointBest = ModelCheckpoint('best_aumentation_mel_7.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='auto', save_freq='epoch') 
checkpointLast = ModelCheckpoint('last_aumentation_mel_7.h5', monitor='val_loss', verbose=1, save_best_only=False, mode='auto', save_freq='epoch') 

In [ ]:
model = models.load_model(path_save+"last_aumentation_mel_6.h5")

In [ ]:
#treinando o modelo
model.fit(x_train, y_train, validation_split=validation_split, batch_size=batch_size, epochs=epochs, callbacks=[checkpointBest, checkpointLast], verbose=1)

Epoch 1/5
164/164 [==============================] - ETA: 0s - loss: 0.1428 - accuracy: 0.9505 
Epoch 00001: val_loss improved from inf to 8.57746, saving model to best_aumentation_mel_7.h5

Epoch 00001: saving model to last_aumentation_mel_7.h5
164/164 [==============================] - 3670s 22s/step - loss: 0.1428 - accuracy: 0.9505 - val_loss: 8.5775 - val_accuracy: 0.2295
Epoch 2/5
164/164 [==============================] - ETA: 0s - loss: 0.1332 - accuracy: 0.9538 
Epoch 00002: val_loss did not improve from 8.57746

Epoch 00002: saving model to last_aumentation_mel_7.h5
164/164 [==============================] - 3654s 22s/step - loss: 0.1332 - accuracy: 0.9538 - val_loss: 10.4617 - val_accuracy: 0.2412
Epoch 3/5
164/164 [==============================] - ETA: 0s - loss: 0.1277 - accuracy: 0.9567 
Epoch 00003: val_loss did not improve from 8.57746

Epoch 00003: saving model to last_aumentation_mel_7.h5
164/164 [==============================] - 3609s 22s/step - loss: 0.1277 - accu

In [ ]:
#Salvando o último modelo 
model.save(path_save+'best_aumentation_mel_7.h5')
model.save(path_save+'last_aumentation_mel_7.h5')